# 1997, Dynamic Stochastic General Equilibrium (DSGE) Models, Finn Kydland and Edward Prescott (Nobel Prize 2004)
Boulouma, A.

To forecast future data based on time index only, we can use a Dynamic Stochastic General Equilibrium (DSGE) model developed by Finn Kydland and Edward Prescott, which won them the Nobel Prize in Economics in 2004.

The model consists of multiple equations:

Market demand for Bitcoin:
$$Q_{d,t} = f(P_B, OMS, TV)$$

where $Q_d$ is the quantity of Bitcoin demanded, $P_B$ is the price of Bitcoin, $OMS$ is overall market sentiment, $TV$ is the transaction volume on the Bitcoin network

Supply of Bitcoin:
$$Q_{d,t} = g(P_B, MC, OMS)$$

where $Q_s$ is the quantity of Bitcoin supplied, $MC$ is the mining difficulty and hash rate and other variables are as defined above.

Equilibrium condition:
$$Q_{d,t} = Q_{s,t}$$

The Bitcoin price at time $t$ can be computed using the equilibrium condition and the supply equation:
$$P_{B,t} = \frac{M_t}{Q_{d,t}} = \frac{M_t}{Q_{s,t}}$$

A time series model can then be used to estimate the coefficients of the demand


In [33]:

from scipy.optimize import minimize
from scipy.optimize import fsolve
from scipy.optimize import brentq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.api import VAR

from btc_analysis import *
from btc_data import *
# Load the data
df = btc_df = clean_and_transform_data(read_data("../datasets/public/btc.csv"), read_data("../datasets/public/btc_google_trend.csv"))

# Set the date column as the index
df.set_index('time', inplace=True)

# df.columns
# Index(['Price', 'hash_rate', 'transaction_volume', 'mining_difficulty','inflation_rate', 'bitcoin_trend'], dtype='object')


/var/folders/gc/m0hv5jzd1kn2nrn5y5w241g00000gn/T/ipykernel_83661/2950300481.py:15: DtypeWarning: Columns (146) have mixed types.Specify dtype option on import or set low_memory=False.
  df = btc_df = clean_and_transform_data(read_data("../datasets/public/btc.csv"), read_data("../datasets/public/btc_google_trend.csv"))


In [36]:
# Define the market demand function
def f(price, oms, tv):
    return np.exp(-price) * oms**0.5 * np.log(tv)

# Define the market supply function
def g(price, mc, ct, oms):
    return price**2 * mc * ct * oms**0.5

def market_demand(price, oms, tv):
    return f(price, oms, tv)

def market_supply(price, mc, ct, oms):
    return g(price, mc, ct, oms)

# Define the equilibrium function
def equilibrium(price, mc, ct, oms, tv):
    return f(price, oms, tv) - g(price, mc, ct, oms)

# Define the variables for the DSGE model
pb = df['Price']
mc = df['mining_difficulty']
ct = df['inflation_rate']
oms = df['bitcoin_trend']
tv = df['transaction_volume']

# Estimate the coefficients of the demand equation using a VAR model
model = VAR(df[['Price', 'transaction_volume', 'bitcoin_trend']])
results = model.fit(2) # choose the lag order based on information criteria or cross-validation
coefficients = results.params['Price']

def forecast_price(t, pb, mc, ct, oms, tv, coefficients):
    # Solve for the equilibrium price using the supply equation and the estimated demand coefficients
    price = brentq(equilibrium, 0, 10000, args=(mc[t], ct[t], oms[t], tv[t]))
    # Compute the forecasted price using the estimated demand coefficients
    forecast_price = np.dot([1, oms[t], tv[t], pb[t-1], mc[t], ct[t]], coefficients[1:])
    return forecast_price

for t in range(len(pb)):
    forecasted_price = forecast_price(t, pb, mc, ct, oms, tv, coefficients)
    print(f"Forecasted price for time {t}: {forecasted_price:.2f} USD")



/Users/macbook/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'


Forecasted price for time 0: 96666.32 USD
Forecasted price for time 1: 212571.04 USD
Forecasted price for time 2: 78479.92 USD
Forecasted price for time 3: 107859.37 USD
Forecasted price for time 4: 126323.95 USD
Forecasted price for time 5: 76286.39 USD
Forecasted price for time 6: 140493.42 USD
Forecasted price for time 7: 102256.03 USD
Forecasted price for time 8: 281113.62 USD
Forecasted price for time 9: 90568.81 USD
Forecasted price for time 10: 248439.52 USD
Forecasted price for time 11: 71196.37 USD
Forecasted price for time 12: 77183.29 USD
Forecasted price for time 13: 56025.34 USD
Forecasted price for time 14: 268780.65 USD
Forecasted price for time 15: 247739.45 USD
Forecasted price for time 16: 125166.02 USD
Forecasted price for time 17: 53419.76 USD
Forecasted price for time 18: 120756.30 USD
Forecasted price for time 19: 72190.10 USD
Forecasted price for time 20: 57675.52 USD
Forecasted price for time 21: 38978.81 USD
Forecasted price for time 22: 307991.45 USD
Forecaste

Forecasted price for time 623: 38089557.26 USD
Forecasted price for time 624: 65626559.44 USD
Forecasted price for time 625: 65182245.72 USD
Forecasted price for time 626: 77252340.67 USD
Forecasted price for time 627: 64319892.55 USD
Forecasted price for time 628: 57255488.36 USD
Forecasted price for time 629: 44791903.69 USD
Forecasted price for time 630: 35964794.87 USD
Forecasted price for time 631: 133284658.84 USD
Forecasted price for time 632: 86338154.52 USD
Forecasted price for time 633: 85023832.15 USD
Forecasted price for time 634: 89461046.98 USD
Forecasted price for time 635: 93523412.69 USD
Forecasted price for time 636: 64955721.04 USD
Forecasted price for time 637: 52746398.80 USD
Forecasted price for time 638: 95359826.93 USD
Forecasted price for time 639: 80351829.51 USD
Forecasted price for time 640: 85577422.05 USD
Forecasted price for time 641: 87855590.67 USD
Forecasted price for time 642: 103512580.65 USD
Forecasted price for time 643: 66646110.42 USD
Forecasted 

Forecasted price for time 1202: 2738687253.99 USD
Forecasted price for time 1203: 2202376221.69 USD
Forecasted price for time 1204: 2373598230.03 USD
Forecasted price for time 1205: 5880940706.69 USD
Forecasted price for time 1206: 7075758893.68 USD
Forecasted price for time 1207: 5773407801.57 USD
Forecasted price for time 1208: 9503587208.71 USD
Forecasted price for time 1209: 9258762626.55 USD
Forecasted price for time 1210: 8228125151.33 USD
Forecasted price for time 1211: 6079989265.58 USD
Forecasted price for time 1212: 5570819652.23 USD
Forecasted price for time 1213: 7319973903.75 USD
Forecasted price for time 1214: 9387328447.16 USD
Forecasted price for time 1215: 10629119856.47 USD
Forecasted price for time 1216: 8441818109.18 USD
Forecasted price for time 1217: 7696539284.91 USD
Forecasted price for time 1218: 8972222889.51 USD
Forecasted price for time 1219: 21517873881.26 USD
Forecasted price for time 1220: 21630128567.25 USD
Forecasted price for time 1221: 19738737747.72 

Forecasted price for time 1605: 5844720360.82 USD
Forecasted price for time 1606: 4262294461.62 USD
Forecasted price for time 1607: 4742252597.94 USD
Forecasted price for time 1608: 3835628372.09 USD
Forecasted price for time 1609: 3260956857.48 USD
Forecasted price for time 1610: 2674715927.12 USD
Forecasted price for time 1611: 4179836746.15 USD
Forecasted price for time 1612: 4752829723.45 USD
Forecasted price for time 1613: 5633380531.02 USD
Forecasted price for time 1614: 6052756232.65 USD
Forecasted price for time 1615: 4712911874.32 USD
Forecasted price for time 1616: 4200188772.93 USD
Forecasted price for time 1617: 4225271373.45 USD
Forecasted price for time 1618: 4975577639.52 USD
Forecasted price for time 1619: 6718993614.47 USD
Forecasted price for time 1620: 3340166384.68 USD
Forecasted price for time 1621: 2375663044.01 USD
Forecasted price for time 1622: 3169906814.00 USD
Forecasted price for time 1623: 3141881386.91 USD
Forecasted price for time 1624: 2600646988.32 USD


...

Forecasted price for time 2860: 200133045994.31 USD
Forecasted price for time 2861: 178582130194.93 USD
Forecasted price for time 2862: 75376654414.64 USD
Forecasted price for time 2863: 83395341747.48 USD
Forecasted price for time 2864: 109244345707.52 USD
Forecasted price for time 2865: 113217972874.98 USD
Forecasted price for time 2866: 135963661564.97 USD
Forecasted price for time 2867: 143489888192.64 USD
Forecasted price for time 2868: 125430799721.36 USD
Forecasted price for time 2869: 68315948136.38 USD
Forecasted price for time 2870: 72431141039.25 USD
Forecasted price for time 2871: 136415362787.00 USD
Forecasted price for time 2872: 132981432987.08 USD
Forecasted price for time 2873: 120199799380.38 USD
Forecasted price for time 2874: 109618900679.53 USD
Forecasted price for time 2875: 112273572779.15 USD
Forecasted price for time 2876: 89970603466.88 USD
Forecasted price for time 2877: 80476212598.13 USD
Forecasted price for time 2878: 98571394438.61 USD
Forecasted price fo

Forecasted price for time 3144: 88558823888.16 USD
Forecasted price for time 3145: 77741949229.16 USD
Forecasted price for time 3146: 81053662520.59 USD
Forecasted price for time 3147: 103144526760.14 USD
Forecasted price for time 3148: 73180120044.85 USD
Forecasted price for time 3149: 56867065025.55 USD
Forecasted price for time 3150: 44385503167.06 USD
Forecasted price for time 3151: 68591593488.39 USD
Forecasted price for time 3152: 77425814805.76 USD
Forecasted price for time 3153: 78499023511.66 USD
Forecasted price for time 3154: 75542425487.07 USD
Forecasted price for time 3155: 78241185426.65 USD
Forecasted price for time 3156: 55216947923.37 USD
Forecasted price for time 3157: 46291680049.90 USD
Forecasted price for time 3158: 72183136826.79 USD
Forecasted price for time 3159: 83321861807.40 USD
Forecasted price for time 3160: 75413514165.15 USD
Forecasted price for time 3161: 84117286812.90 USD
Forecasted price for time 3162: 74547105025.49 USD
Forecasted price for time 3163

Forecasted price for time 3468: 186774465200.89 USD
Forecasted price for time 3469: 147387543831.26 USD
Forecasted price for time 3470: 156455921200.99 USD
Forecasted price for time 3471: 86629848936.50 USD
Forecasted price for time 3472: 79315676498.89 USD
Forecasted price for time 3473: 132896363579.81 USD
Forecasted price for time 3474: 107622610530.91 USD
Forecasted price for time 3475: 119061611161.60 USD
Forecasted price for time 3476: 135895300608.16 USD
Forecasted price for time 3477: 94370855430.57 USD
Forecasted price for time 3478: 63814814281.79 USD
Forecasted price for time 3479: 60032083918.03 USD
Forecasted price for time 3480: 126114764400.90 USD
Forecasted price for time 3481: 127407458329.90 USD
Forecasted price for time 3482: 124597335890.31 USD
Forecasted price for time 3483: 158103989915.70 USD
Forecasted price for time 3484: 158643703642.13 USD
Forecasted price for time 3485: 107100316999.82 USD
Forecasted price for time 3486: 73118606371.28 USD
Forecasted price f

Forecasted price for time 3979: 907643507562.06 USD
Forecasted price for time 3980: 552912145654.67 USD
Forecasted price for time 3981: 599507742123.50 USD
Forecasted price for time 3982: 335325584126.89 USD
Forecasted price for time 3983: 371474811812.96 USD
Forecasted price for time 3984: 724330249634.12 USD
Forecasted price for time 3985: 970152159702.42 USD
Forecasted price for time 3986: 790480751672.44 USD
Forecasted price for time 3987: 736524922308.72 USD
Forecasted price for time 3988: 652568142164.46 USD
Forecasted price for time 3989: 370035879015.02 USD
Forecasted price for time 3990: 404871214937.78 USD
Forecasted price for time 3991: 845378594640.46 USD
Forecasted price for time 3992: 745756174382.98 USD
Forecasted price for time 3993: 760920532183.89 USD
Forecasted price for time 3994: 693376882413.18 USD
Forecasted price for time 3995: 761730829640.38 USD
Forecasted price for time 3996: 608955245580.93 USD
Forecasted price for time 3997: 438620841097.47 USD
Forecasted p

Forecasted price for time 4483: 556768040661.13 USD
Forecasted price for time 4484: 381249717518.42 USD
Forecasted price for time 4485: 324189122130.82 USD
Forecasted price for time 4486: 315724912484.39 USD
Forecasted price for time 4487: 215507611405.51 USD
Forecasted price for time 4488: 370118086310.58 USD
Forecasted price for time 4489: 400262708941.73 USD
Forecasted price for time 4490: 395301081742.03 USD
Forecasted price for time 4491: 308177378450.61 USD
Forecasted price for time 4492: 511794323455.10 USD
Forecasted price for time 4493: 418077557078.08 USD
Forecasted price for time 4494: 172853714469.72 USD
Forecasted price for time 4495: 346645683439.76 USD
Forecasted price for time 4496: 425269737890.41 USD
Forecasted price for time 4497: 528608738405.80 USD
Forecasted price for time 4498: 512993888509.44 USD
Forecasted price for time 4499: 388817783347.67 USD
Forecasted price for time 4500: 301182835425.40 USD
Forecasted price for time 4501: 265817306099.95 USD
Forecasted p